In [ ]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm
from dateutil.parser import parse
import os

os.makedirs("data/raw", exist_ok=True)

In [ ]:
# Import raw news data
news_df = pd.read_csv("data/raw/sp500_news_290k_articles.csv")

In [ ]:
def safe_parse_date(x):
    try:
        dt = parse(x)
        return dt.replace(tzinfo=None)
    except Exception:
        return pd.NaT

# Apply safe, timezone-free parsing across column
news_df["date"] = news_df["date"].astype(str).apply(safe_parse_date)

# Drop invalid dates
news_df = news_df.dropna(subset=["date"])

In [ ]:
#Clean ticker data and drop empty columns
news_df["ticker"] = news_df["ticker"].str.upper()
news_df = news_df.dropna(subset=["date", "headline", "ticker"])

In [ ]:
#Group the news headlines by ticker and date
grouped = news_df.groupby(["date", "ticker"]).agg({
    "headline": lambda x: " ".join(x),
    "compound": "mean"
}).reset_index()
grouped.rename(columns={
    "headline": "combined_headlines",
    "compound": "avg_sentiment"
}, inplace=True)

In [33]:
grouped.head()

,date,ticker,combined_headlines,avg_sentiment
0,2019-12-19,L,Cordish Cos. leader confident in office projec...,0.000000
1,2020-01-02,L,"Despite Its High P/E Ratio, Is Loews Corporati...",0.000000
2,2020-01-22,L,Loews Corporation to Release Fourth Quarter 20...,0.354325
3,2020-02-07,L,"Altium Packaging, LLC -- Moody's assigns Altiu...",0.378950
4,2020-02-10,L,"Loews (L) Q4 Earnings Beat Estimates, Revenues...",0.236487


In [ ]:
# Export the aggregated news headlines into its own csv file
grouped.to_csv("data/raw/news_aggregated.csv", index=False)

In [ ]:
# Import the aggregated news data and group the 
news_grouped = pd.read_csv("data/raw/news_aggregated.csv")
tickers = news_grouped["ticker"].unique().tolist()
tickers = [ticker for ticker in tickers if isinstance(ticker, str) and ticker.isalpha()]

In [ ]:
# Get the price data for all the tickers in the aggregated headlines file
def get_price_data(ticker, start="2019-12-01", end="2023-03-05"):
    try:
        df = yf.Ticker(ticker).history(start=start, end=end)
        if df.empty:
            return None
        df = df.reset_index()
        df["ticker"] = ticker
        return df[["Date", "Open", "High", "Low", "Close", "Volume", "ticker"]]
    except Exception as e:
        print(f"Error downloading {ticker}: {e}")
        return None

In [ ]:
# Save all the prices in a list from Yahoo Finance API data
# Error handle for the failed tickers that don't show up
all_prices = []
failed_tickers = []

for t in tqdm(tickers):
    data = get_price_data(t)
    if data is not None and not data.empty:
        all_prices.append(data)
    else:
        failed_tickers.append(t)

100%|██████████| 496/496 [01:56<00:00,  4.26it/s]


In [ ]:
# Combines stock prices into a dataframe
price_df = pd.concat(all_prices, ignore_index=True)

# Renames and reformats the data into ingestible format
price_df.rename(columns={"Date": "date"}, inplace=True)
price_df["date"] = pd.to_datetime(price_df["date"], errors='coerce')

# Drop empty rows and sort the values
price_df = price_df.dropna().sort_values(["ticker", "date"]).reset_index(drop=True)

In [ ]:
# Ensure the shape and outputs of the new dataframe is accurate
print(price_df.shape)
print(price_df.head())

(378025, 7)
                       date       Open       High        Low      Close  \
0 2019-12-02 00:00:00-05:00  77.572081  77.773741  76.842260  77.159157   
1 2019-12-03 00:00:00-05:00  76.362129  76.928703  76.026029  76.919098   
2 2019-12-04 00:00:00-05:00  77.111139  77.783338  76.995902  77.716118   
3 2019-12-05 00:00:00-05:00  77.677706  78.493949  77.303194  78.292290   
4 2019-12-06 00:00:00-05:00  78.974097  79.146949  78.570778  78.945290   

    Volume ticker  
0  1775600      A  
1  1978200      A  
2  1690900      A  
3  1900000      A  
4  1783400      A  


In [ ]:
# Removes timezone info from the date column
price_df["date"] = price_df["date"].dt.tz_localize(None)

# Creates a binary target of 1 if the next closing is greater
# than today's closing
price_df["target"] = (
    price_df.groupby("ticker")["Close"].shift(-1) > price_df["Close"]
).astype(int)

In [ ]:
# Grouping the news by date and merging the prices and news data
news_grouped["date"] = pd.to_datetime(news_grouped["date"], errors="coerce")
merged_df = pd.merge(price_df, news_grouped, on=["date", "ticker"], how="left")

In [ ]:
#Dropping any null values
merged_df = merged_df.dropna(subset=["avg_sentiment"]).reset_index(drop=True)

In [ ]:
# Ensure that the final dataframe looks right
print(merged_df.shape)
print(merged_df.dtypes)
merged_df.head()

(84052, 10)
date                  datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close                        float64
Volume                         int64
ticker                        object
target                         int64
combined_headlines            object
avg_sentiment                float64
dtype: object


,date,Open,High,Low,Close,Volume,ticker,target,combined_headlines,avg_sentiment
0,2021-08-16,155.066666,158.111772,154.200807,157.673981,2154300,A,0,Factors to Note Ahead of Keysight's (KEYS) Q3 ...,0.000000
1,2021-08-17,157.343206,157.450223,154.969387,156.545456,2614200,A,1,Agilent Technologies (A) Q3 Earnings and Reven...,-0.148780
2,2021-08-18,158.773301,160.874716,157.693408,158.598190,2860900,A,1,Agilent (A) Q3 Earnings & Revenues Beat Estima...,-0.041667
3,2021-08-19,157.352924,164.493829,157.226446,163.122070,2179600,A,1,Researchers Develop COVID-19 Severity Screenin...,0.000000
4,2021-08-23,164.834324,165.943417,163.569597,165.388870,2061400,A,1,Agilent Companion Diagnostic Expands CE-IVD Ma...,-0.306200


In [ ]:
# Ensuring data quality seems appropriate
merged_df["ticker"].value_counts().describe()

count    462.000000
mean     181.930736
std       24.579579
min       88.000000
25%      167.250000
50%      185.000000
75%      199.000000
max      256.000000
Name: count, dtype: float64

In [ ]:
# Saved the merged data in a new csv
merged_df.to_csv("data/processed/merged_news_stock_dataset.csv", index=False)